## Estimated vote share

Build a model to predict out an estimated Labour vote share in the dataset included in this email. 

Fill in the missing election results in the CSV file (those marked NA in column B), and attach an explanation of how you achieved this. 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import (
    GradientBoostingRegressor,
    RandomForestRegressor,
    VotingRegressor,
)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


In [3]:
pd.set_option('display.max_columns', None)
df_all= pd.read_csv('data.csv')

In [4]:
df_all.head(10)

,wpc,LAB_19,region,Winner_19,LAB_17,Winner_17,Population.density,Age_under_18,Age_18_to_24,Age_25_to_44,Age_45_to_64,Age_65_and_plus,median_age,Born.in.UK,Unemployed,Retired,Economically.Inactive,Degree,Diversity,Education,AsianPC,BlackPC,BritishOrIrishPC,JewishPC,OtherEuropeanPC,A.agriculture.and.fishing,B.D.E.energy.and.water,C.manufacturing,F.construction,G.I.distribution..hotels.and.restaurants,H.J.transport.and.communications,K.N.banking..finance.and.insurance,O.Q.public.admin..education.and.health,R.U.other.services,G.Q.total.services,Estimated.Leave.proportion,Anomie,Public.sector,Urban.rural,Poor.health,Distance.cosmopolian.city,Distance.university,Catholic.schools
0,WPC_1,7.9,South East,Conservative,0.1259,Conservative,4.023521,0.224762,0.064638,0.249319,0.270711,0.190569,41.920511,89.168905,2.484600,14.127021,26.385849,28.035015,-0.032198,0.196122,0.018034,0.003355,0.880778,0.001928,0.095905,0.0,2.6,8.9,5.3,11.3,15.3,26.4,21.6,7.4,81.9,0.406079,0.296325,7.201664,semi-rural/rural,3.263873,0.581343,0.131558,8.333333
1,WPC_2,7.7,Eastern,Conservative,0.0993,LibDem,1.163616,0.162268,0.062000,0.178540,0.298545,0.298648,51.933864,95.970229,3.369165,24.659102,37.843801,10.311081,-0.114537,-0.005407,0.004571,0.001600,0.921625,0.001043,0.071161,10.6,0.0,5.3,5.4,25.7,5.9,8.9,26.2,8.7,75.6,0.582539,0.315946,12.806969,NaN,6.331148,0.222059,0.225604,0.000000
2,WPC_3,67.0,East Midlands,Labour,0.7355,Labour,56.341334,0.229194,0.169704,0.290822,0.200038,0.110242,31.528251,66.179885,5.802527,8.017470,39.776500,19.527121,0.421376,-0.172921,0.385019,0.029043,0.476649,0.001606,0.107682,0.0,1.7,12.6,4.8,21.3,6.0,16.4,33.4,1.5,78.5,0.425820,0.357568,21.528738,Large urban,5.466648,0.756731,0.009915,4.651163
3,WPC_4,NaN,South East,Conservative,0.0000,Other,1.120997,0.222282,0.062849,0.229173,0.307998,0.177698,43.712408,92.702502,2.346535,14.558699,26.612447,22.673744,-0.059310,0.177016,0.016552,0.004273,0.896970,0.002291,0.079914,1.8,0.0,10.7,9.1,13.6,5.4,20.1,32.6,6.1,77.8,0.486547,0.266156,8.379839,semi-rural/rural,3.200000,0.468253,0.141594,3.030303
4,WPC_5,7.0,South West,Conservative,0.1153,Conservative,1.858880,0.206289,0.071584,0.212975,0.291459,0.217693,NaN,94.256623,2.884550,16.974938,30.268189,14.507132,-0.089281,0.077575,0.009133,0.002672,0.911957,0.001229,0.075008,2.3,0.0,7.4,7.2,22.4,7.0,13.6,33.9,5.4,82.4,0.534931,0.315365,9.349402,NaN,5.337837,0.528522,0.269327,5.555556
5,WPC_6,48.6,South East,Labour,0.4104,Labour,80.059358,0.177599,0.217673,0.295024,0.188867,0.120836,31.003827,83.638543,4.370703,19.952919,33.064504,18.220114,0.070221,0.037618,0.063088,0.015395,0.800462,0.002167,0.118888,0.0,1.5,9.0,4.6,19.2,9.7,15.9,36.0,3.8,84.6,0.517630,0.523248,19.462095,NaN,4.933523,1.114877,0.013474,12.500000
6,WPC_7,32.0,Eastern,Conservative,0.3113,Conservative,10.733728,0.219962,0.085779,0.276050,0.250772,0.167437,39.410508,89.610623,5.745456,13.445750,30.388614,9.894572,-0.047028,-0.092806,0.037750,0.014515,0.836523,0.004095,0.107116,0.0,1.3,11.6,4.9,16.9,6.8,18.5,31.2,8.3,81.6,0.604552,0.389804,17.229553,NaN,6.128913,0.723885,0.319601,11.764706
7,WPC_8,20.8,Eastern,Conservative,0.3113,Conservative,14.891865,0.228241,0.078066,0.269764,0.254288,0.169641,39.407731,92.506160,4.613724,14.280797,29.276573,10.881236,-0.067074,-0.023930,0.027933,0.016725,0.859601,0.001917,0.093824,0.0,0.0,10.2,6.7,8.3,7.2,34.4,28.9,3.1,81.9,0.669166,0.290129,11.846488,NaN,5.230412,0.308203,0.143153,12.903226
8,WPC_9,14.3,South East,Conservative,0.2068,Conservative,1.466673,0.209095,0.068161,0.255171,0.280017,0.187556,42.671025,91.901049,2.271444,13.678558,23.183173,20.873968,-0.074789,0.102369,0.012132,0.002444,0.902898,0.001575,0.080952,0.0,0.0,11.8,5.6,19.1,10.9,16.3,28.9,5.1,80.2,0.463447,0.311267,10.391980,semi-rural/rural,3.504519,0.747710,0.257460,4.615385
9,WPC_10,48.6,Yorkshire and Humber,Labour,0.4411,Labour,13.563913,0.155419,0.245462,0.227950,0.219483,0.151686,36.470993,91.356375,3.132451,11.463660,35.031265,24.565286,0.029426,0.175582,0.059664,0.0

In [5]:
# LAB_19 should be a proportion like LA17 so divide by 100
df_all['LAB_19'] = df_all['LAB_19']/100

In [6]:
df_all[df_all['LAB_19'].isna()].head(10)  # OK can use LAB_17 as a feature

,wpc,LAB_19,region,Winner_19,LAB_17,Winner_17,Population.density,Age_under_18,Age_18_to_24,Age_25_to_44,Age_45_to_64,Age_65_and_plus,median_age,Born.in.UK,Unemployed,Retired,Economically.Inactive,Degree,Diversity,Education,AsianPC,BlackPC,BritishOrIrishPC,JewishPC,OtherEuropeanPC,A.agriculture.and.fishing,B.D.E.energy.and.water,C.manufacturing,F.construction,G.I.distribution..hotels.and.restaurants,H.J.transport.and.communications,K.N.banking..finance.and.insurance,O.Q.public.admin..education.and.health,R.U.other.services,G.Q.total.services,Estimated.Leave.proportion,Anomie,Public.sector,Urban.rural,Poor.health,Distance.cosmopolian.city,Distance.university,Catholic.schools
3,WPC_4,NaN,South East,Conservative,0.0000,Other,1.120997,0.222282,0.062849,0.229173,0.307998,0.177698,43.712408,92.702502,2.346535,14.558699,26.612447,22.673744,-0.059310,0.177016,0.016552,0.004273,0.896970,0.002291,0.079914,1.8,0.0,10.7,9.1,13.6,5.4,20.1,32.6,6.1,77.8,0.486547,0.266156,8.379839,semi-rural/rural,3.200000,0.468253,0.141594,3.030303
61,WPC_62,NaN,London,Conservative,0.4223,Labour,130.707125,0.176245,0.092353,0.390284,0.229418,0.111700,NaN,47.682345,4.547112,7.672514,31.055668,46.268415,0.454816,0.171328,0.155938,0.032507,0.491650,0.009475,0.310430,0.0,0.0,1.7,3.5,9.5,7.7,41.6,21.2,12.4,92.4,0.313504,0.474267,9.793329,NaN,4.738544,0.085459,0.011269,14.754098
78,WPC_79,NaN,South East,Conservative,0.2216,Conservative,1.780965,0.211972,0.066947,0.214524,0.295316,0.211241,44.827941,92.603480,2.609645,16.532821,28.888551,16.681421,-0.086483,0.096221,0.011649,0.002983,0.900007,0.001804,0.083556,0.0,0.0,9.2,8.8,20.3,5.4,15.7,30.0,8.6,80.1,0.525463,0.271898,11.342854,semi-rural/rural,4.106657,0.533102,0.294385,7.272727
89,WPC_90,NaN,London,Labour,0.6085,Labour,58.332540,0.231119,0.091883,0.333515,0.223124,0.120359,34.703392,53.348054,4.688395,9.289582,28.841048,28.940843,0.559536,-0.106045,0.469026,0.027242,0.351117,0.004101,0.148514,0.0,0.0,5.3,10.7,16.0,13.4,20.6,29.1,4.4,83.4,0.450562,0.315106,12.900244,NaN,4.317189,0.087567,0.087116,11.111111
97,WPC_98,NaN,Eastern,Conservative,0.2043,Conservative,2.935099,0.209279,0.070123,0.245091,0.281275,0.194232,42.975704,92.427682,3.101053,15.140787,28.199268,16.772827,-0.051900,0.046128,0.034433,0.008349,0.860259,0.003023,0.093938,5.2,0.0,5.3,9.0,15.9,7.9,29.4,18.6,8.6,80.5,0.608937,0.305561,7.484035,NaN,4.118033,0.187794,0.218526,10.416667
100,WPC_101,NaN,West Midlands,Labour,0.6812,Labour,43.456031,0.262630,0.111277,0.291455,0.208921,0.125717,33.324255,70.195163,7.812335,10.162118,34.233841,12.263932,0.362468,-0.280675,0.407263,0.030972,0.485704,0.001068,0.074993,0.0,0.0,10.5,5.7,9.7,8.6,18.1,33.3,12.2,81.9,0.512173,0.304244,19.919563,Large urban,6.571108,0.053657,0.018223,17.021277
102,WPC_103,NaN,East Midlands,Conservative,0.2670,Conservative,0.703936,0.174330,0.065370,0.190130,0.303106,0.267065,NaN,96.676472,3.584315,24.548965,39.120201,8.648622,-0.148361,-0.067355,0.005618,0.001904,0.932687,0.000833,0.058958,8.5,0.0,8.5,6.1,21.9,12.5,13.6,24.9,3.5,76.3,0.694253,0.315249,13.797587,semi-rural/rural,7.821644,0.566453,0.546846,0.000000
110,WPC_111,NaN,South East,Conservative,0.2094,Conservative,3.743381,0.224567,0.066410,0.244155,0.279673,0.185195,42.297529,91.395789,2.822986,14.127021,26.385849,21.003653,-0.057744,0.096332,0.020353,0.007233,0.876338,0.002222,0.093854,0.0,0.0,7.2,8.1,16.2,11.2,23.5,30.6,0.0,84.7,0.538611,0.269874,6.633514,NaN,3.986899,0.238725,0.229375,3.846154
117,WPC_118,NaN,North West,Conservative,0.5328,Labour,20.093112,0.227120,0.080843,0.270713,0.262632,0.158691,39.353506,91.005907,4.351018,13.870458,29.172431,15.644448,0.005694,-0.004288,0.060088,0.007118,0.815943,0.024526,0.092325,0.0,2.4,9.4,8.4,19.7,5.2,16.7,32.1,5.4,79.1,0.545260,0.299546,19.914000,Large urban,6.071012,0.534597,0.080308,11.904762
132,WPC_133,NaN,Yorkshire and Humber,Labour,0.5307,Labour,35.670314,0.199584,0.097686,0.287247,0.253631,0.161852,38.977423,89.726658,8.036619,9.630719,26.755992,8.599960,-0.076657,-0.16

In [7]:
# How many missing values are there?
sum(df_all['LAB_19'].isna())

16

In [8]:
df_all.describe()

,LAB_19,LAB_17,Population.density,Age_under_18,Age_18_to_24,Age_25_to_44,Age_45_to_64,Age_65_and_plus,median_age,Born.in.UK,Unemployed,Retired,Economically.Inactive,Degree,Diversity,Education,AsianPC,BlackPC,BritishOrIrishPC,JewishPC,OtherEuropeanPC,A.agriculture.and.fishing,B.D.E.energy.and.water,C.manufacturing,F.construction,G.I.distribution..hotels.and.restaurants,H.J.transport.and.communications,K.N.banking..finance.and.insurance,O.Q.public.admin..education.and.health,R.U.other.services,G.Q.total.services,Estimated.Leave.proportion,Anomie,Public.sector,Poor.health,Distance.cosmopolian.city,Distance.university,Catholic.schools
count,327.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,313.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000,343.000000
mean,0.344391,0.421957,20.997766,0.211549,0.088154,0.269508,0.259743,0.171046,40.350774,87.708770,4.301943,15.205678,30.355641,16.758932,0.007644,0.002702,0.068924,0.011534,0.819789,0.002414,0.097339,0.747813,1.201166,9.872886,7.107872,18.648980,8.839942,16.794461,29.681341,5.699125,79.769971,0.537341,0.321098,17.392299,5.603447,0.465889,0.159790,8.619619
std,0.174255,0.178096,26.632927,0.022538,0.030040,0.049471,0.032581,0.042924,4.389038,11.485710,1.440291,4.222036,3.921617,8.865200,0.181968,0.135384,0.099446,0.017007,0.143533,0.004401,0.053147,1.960409,1.569668,4.672327,2.843536,4.957349,3.784163,6.353049,5.629512,2.558537,6.067621,0.109152,0.056978,7.400294,1.621962,0.330851,0.138984,6.318290
min,0.044000,0.000000,0.263601,0.125730,0.058348,0.174263,0.136829,0.063658,28.510147,44.450154,1.837399,5.229428,19.192094,5.098757,-0.176543,-0.337479,0.004014,0.000810,0.214321,0.000462,0.039941,0.000000,0.000000,0.000000,0.000000,5.700000,0.000000,5.700000,11.300000,0.000000,61.600000,0.205397,0.236740,5.086069,2.662342,0.010656,0.002902,0.000000
25%,0.208000,0.273750,3.078909,0.198634,0.071600,0.239933,0.241865,0.149358,37.721024,85.637403,3.083166,12.581298,27.474331,10.827010,-0.106638,-0.078699,0.011329,0.002652,0.792201,0.001108,0.064796,0.000000,0.000000,6.900000,5.400000,15.250000,6.200000,12.250000,26.000000,4.200000,75.950000,0.486995,0.286622,11.761445,4.319094,0.192082,0.057453,4.081633
50%,0.344000,0.417800,10.291853,0.210656,0.080783,0.262340,0.267660,0.169392,40.578742,91.653000,4.035019,15.140787,30.050492,14.530523,-0.063646,0.005835,0.026120,0.004872,0.874926,0.001537,0.079892,0.000000,0.000000,9.600000,7.000000,18.500000,8.600000,15.800000,29.400000,5.400000,79.700000,0.551578,0.307740,16.289726,5.259655,0.410803,0.125736,7.547170
75%,0.463500,0.567350,30.865794,0.223934,0.092295,0.286692,0.282766,0.197879,43.309521,95.157708,5.153441,18.619030,33.064504,19.489905,0.038568,0.089163,0.088021,0.012993,0.915897,0.002080,0.107399,0.000000,2.300000,12.450000,8.550000,21.950000,11.200000,19.700000,33.550000,7.200000,83.250000,0.608869,0.334609,21.654040,6.689019,0.676667,0.226341,12.000000
max,0.847000,0.857300,146.384606,0.293380,0.312076,0.498292,0.322417,0.329393,52.901525,98.018436,8.633560,25.840831,42.398211,51.098323,0.743299,0.418989,0.559723,0.149444,0.952986,0.063065,0.358165,14.000000,8.500000,29.400000,16.600000,33.800000,23.900000,44.100000,47.500000,13.400000,96.500000,0.749608,0.644770,52.197471,10.691265,1.966113,1.107779,44.736842


In [9]:
mask = df_all['LAB_19'].isna()
df = df_all[~mask].select_dtypes(exclude=['object'])  # exclude non-numeric columns and rows with missing values

In [10]:
df

,LAB_19,LAB_17,Population.density,Age_under_18,Age_18_to_24,Age_25_to_44,Age_45_to_64,Age_65_and_plus,median_age,Born.in.UK,Unemployed,Retired,Economically.Inactive,Degree,Diversity,Education,AsianPC,BlackPC,BritishOrIrishPC,JewishPC,OtherEuropeanPC,A.agriculture.and.fishing,B.D.E.energy.and.water,C.manufacturing,F.construction,G.I.distribution..hotels.and.restaurants,H.J.transport.and.communications,K.N.banking..finance.and.insurance,O.Q.public.admin..education.and.health,R.U.other.services,G.Q.total.services,Estimated.Leave.proportion,Anomie,Public.sector,Poor.health,Distance.cosmopolian.city,Distance.university,Catholic.schools
0,0.079,0.1259,4.023521,0.224762,0.064638,0.249319,0.270711,0.190569,41.920511,89.168905,2.484600,14.127021,26.385849,28.035015,-0.032198,0.196122,0.018034,0.003355,0.880778,0.001928,0.095905,0.0,2.6,8.9,5.3,11.3,15.3,26.4,21.6,7.4,81.9,0.406079,0.296325,7.201664,3.263873,0.581343,0.131558,8.333333
1,0.077,0.0993,1.163616,0.162268,0.062000,0.178540,0.298545,0.298648,51.933864,95.970229,3.369165,24.659102,37.843801,10.311081,-0.114537,-0.005407,0.004571,0.001600,0.921625,0.001043,0.071161,10.6,0.0,5.3,5.4,25.7,5.9,8.9,26.2,8.7,75.6,0.582539,0.315946,12.806969,6.331148,0.222059,0.225604,0.000000
2,0.670,0.7355,56.341334,0.229194,0.169704,0.290822,0.200038,0.110242,31.528251,66.179885,5.802527,8.017470,39.776500,19.527121,0.421376,-0.172921,0.385019,0.029043,0.476649,0.001606,0.107682,0.0,1.7,12.6,4.8,21.3,6.0,16.4,33.4,1.5,78.5,0.425820,0.357568,21.528738,5.466648,0.756731,0.009915,4.651163
4,0.070,0.1153,1.858880,0.206289,0.071584,0.212975,0.291459,0.217693,NaN,94.256623,2.884550,16.974938,30.268189,14.507132,-0.089281,0.077575,0.009133,0.002672,0.911957,0.001229,0.075008,2.3,0.0,7.4,7.2,22.4,7.0,13.6,33.9,5.4,82.4,0.534931,0.315365,9.349402,5.337837,0.528522,0.269327,5.555556
5,0.486,0.4104,80.059358,0.177599,0.217673,0.295024,0.188867,0.120836,31.003827,83.638543,4.370703,19.952919,33.064504,18.220114,0.070221,0.037618,0.063088,0.015395,0.800462,0.002167,0.118888,0.0,1.5,9.0,4.6,19.2,9.7,15.9,36.0,3.8,84.6,0.517630,0.523248,19.462095,4.933523,1.114877,0.013474,12.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,0.363,0.5338,2.292489,0.206511,0.075613,0.241952,0.290788,0.185136,42.988805,97.403092,4.321426,14.127021,26.385849,10.139870,-0.137258,-0.026257,0.006695,0.001222,0.939125,0.000714,0.052244,0.7,1.4,15.7,6.0,17.8,11.0,12.8,30.2,4.3,76.1,0.589488,0.308131,11.210694,7.995244,0.422419,0.167943,10.204082
339,0.548,0.5707,47.503505,0.265294,0.092883,0.280616,0.219427,0.141780,NaN,82.101032,7.286231,11.716791,33.755743,9.220723,0.123208,-0.306308,0.247038,0.012456,0.672756,0.001066,0.066683,0.0,0.0,6.0,8.5,19.5,15.0,13.7,30.5,4.8,83.4,0.600948,0.277969,13.496404,7.157713,0.071343,0.085990,12.765957
340,0.449,0.5446,7.593529,0.226013,0.082795,0.267612,0.264868,0.158713,39.481845,97.255691,5.045756,15.635712,34.608007,11.392369,-0.130021,-0.026680,0.009527,0.001855,0.932375,0.001157,0.055086,0.0,3.3,12.8,7.1,16.8,4.4,14.4,34.0,5.7,75.3,0.551576,0.282654,24.855058,9.005518,0.101090,0.111517,2.272727
341,0.533,0.5916,7.256798,0.202000,0.079753,0.243704,0.286358,0.188184,42.851385,95.915456,3.958525,19.469748,30.050492,12.496144,-0.132450,-0.019418,0.010168,0.002431,0.924005,0.001616,0.061780,0.0,0.0,13.0,7.3,23.8,9.5,17.5,20.6,5.6,76.9,0.577668,0.285594,10.589866,6.134871,0.591412,0.085013,16.216216


In [11]:
df.isna().any()

LAB_19                                      False
LAB_17                                      False
Population.density                          False
Age_under_18                                False
Age_18_to_24                                False
Age_25_to_44                                False
Age_45_to_64                                False
Age_65_and_plus                             False
median_age                                   True
Born.in.UK                                  False
Unemployed                                  False
Retired                                     False
Economically.Inactive                       False
Degree                                      False
Diversity                                   False
Education                                   False
AsianPC                                     False
BlackPC                                     False
BritishOrIrishPC                            False
JewishPC                                    False


In [12]:
sum(df['median_age'].isna())

28

In [13]:
# OK have 28 missing values in median_age
# Would do something clever here with estimating the missing values from the grouped data but for now just drop them all
# Can look at power of this column later which would inform whether to bother doing something clever
mask = df['median_age'].isna()
df = df[~mask]

In [14]:
df.describe()

,LAB_19,LAB_17,Population.density,Age_under_18,Age_18_to_24,Age_25_to_44,Age_45_to_64,Age_65_and_plus,median_age,Born.in.UK,Unemployed,Retired,Economically.Inactive,Degree,Diversity,Education,AsianPC,BlackPC,BritishOrIrishPC,JewishPC,OtherEuropeanPC,A.agriculture.and.fishing,B.D.E.energy.and.water,C.manufacturing,F.construction,G.I.distribution..hotels.and.restaurants,H.J.transport.and.communications,K.N.banking..finance.and.insurance,O.Q.public.admin..education.and.health,R.U.other.services,G.Q.total.services,Estimated.Leave.proportion,Anomie,Public.sector,Poor.health,Distance.cosmopolian.city,Distance.university,Catholic.schools
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000
mean,0.341445,0.420737,20.520524,0.210372,0.088275,0.268795,0.260289,0.172269,40.331680,88.223353,4.268644,15.243083,30.353622,16.681839,0.000475,0.006659,0.064791,0.010867,0.826825,0.002293,0.095224,0.753512,1.307023,9.941806,6.997324,18.737458,8.743478,16.663880,29.849833,5.624749,79.724749,0.537017,0.321473,17.778611,5.612852,0.470076,0.159528,8.659788
std,0.170152,0.173389,26.252570,0.022065,0.030698,0.049255,0.031828,0.042740,4.415708,10.841169,1.404616,4.236101,4.017024,8.965274,0.174669,0.134593,0.093312,0.016710,0.135127,0.004227,0.050924,1.945244,1.618408,4.600134,2.813723,5.006680,3.854847,6.304795,5.666257,2.540581,6.055226,0.110373,0.057833,7.570809,1.632466,0.336706,0.139651,6.456548
min,0.044000,0.091200,0.263601,0.125730,0.058348,0.174263,0.136829,0.067773,28.510147,46.439613,1.837399,5.229428,19.192094,5.098757,-0.176543,-0.337479,0.004014,0.000810,0.268644,0.000462,0.039941,0.000000,0.000000,0.000000,0.000000,5.700000,0.000000,5.700000,11.300000,0.000000,61.600000,0.205397,0.236740,5.086069,2.662342,0.010656,0.002902,0.000000
25%,0.208000,0.285000,3.322933,0.198341,0.071670,0.239536,0.244155,0.149718,37.668116,86.592215,3.108163,12.594638,27.376156,10.775069,-0.109323,-0.074830,0.011305,0.002593,0.800123,0.001108,0.064480,0.000000,0.000000,7.050000,5.350000,15.150000,6.200000,12.100000,26.250000,4.100000,75.850000,0.488750,0.286622,11.852489,4.370129,0.192082,0.057226,4.081633
50%,0.343000,0.410400,10.291853,0.209645,0.080692,0.262754,0.267660,0.169966,40.578742,91.831362,4.018685,15.075951,30.050492,14.378870,-0.065743,0.007620,0.025510,0.004565,0.879320,0.001504,0.078037,0.000000,0.000000,9.600000,6.800000,18.500000,8.400000,15.800000,29.400000,5.400000,79.700000,0.554142,0.307761,16.496018,5.244183,0.405759,0.127999,7.692308
75%,0.454500,0.557400,30.611362,0.223154,0.092128,0.284671,0.282269,0.198286,43.344032,95.260978,5.093206,18.622169,33.064504,19.224726,0.030972,0.089163,0.085423,0.012089,0.916355,0.001981,0.106214,0.000000,2.500000,12.450000,8.500000,22.150000,11.100000,19.500000,33.950000,7.100000,83.150000,0.609484,0.334609,22.299395,6.700740,0.676667,0.220349,12.195122
max,0.847000,0.857300,146.384606,0.293380,0.312076,0.498292,0.322417,0.329393,52.901525,98.018436,8.633560,25.840831,42.398211,51.098323,0.668457,0.418989,0.559723,0.149444,0.952986,0.063065,0.358165,14.000000,8.500000,29.400000,15.900000,33.800000,23.900000,44.100000,47.500000,13.400000,96.500000,0.749608,0.644770,52.197471,10.691265,1.966113,1.107779,44.736842


In [16]:
# Will start with a basic linear model
# Will at least tell use which features are most important
X = df.drop(columns=['LAB_19'])
y = df['LAB_19']

reg = LinearRegression().fit(X, y)
reg.score(X, y)


0.9680209425949952

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [18]:
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_test, y_test)

0.9412560763585116

In [19]:
reg.predict(X_test)

array([0.29441924, 0.21322975, 0.52178559, 0.29583161, 0.29686297,
       0.18012872, 0.20622682, 0.40851033, 0.23092962, 0.55278485,
       0.46342148, 0.42234639, 0.30196746, 0.45527279, 0.1581358 ,
       0.13286383, 0.41706602, 0.10979105, 0.17924231, 0.31871145,
       0.34799428, 0.48551804, 0.67974554, 0.2376356 , 0.42584507,
       0.17220533, 0.30247126, 0.16904753, 0.30670382, 0.36899976])

In [20]:
y_test

322    0.271
306    0.220
191    0.565
11     0.272
90     0.315
319    0.206
112    0.195
131    0.410
7      0.208
201    0.575
118    0.376
225    0.454
214    0.311
179    0.402
66     0.164
70     0.099
172    0.437
130    0.148
73     0.208
165    0.286
180    0.359
126    0.501
284    0.617
54     0.246
50     0.422
316    0.181
321    0.370
140    0.168
245    0.301
270    0.397
Name: LAB_19, dtype: float64

In [21]:
# Let's have a go with our real data 
reg = LinearRegression().fit(X, y)

X_demo = df_all[df_all['LAB_19'].isna()].select_dtypes(exclude=['object']).drop(columns=['LAB_19'])

In [22]:
X_demo

,LAB_17,Population.density,Age_under_18,Age_18_to_24,Age_25_to_44,Age_45_to_64,Age_65_and_plus,median_age,Born.in.UK,Unemployed,Retired,Economically.Inactive,Degree,Diversity,Education,AsianPC,BlackPC,BritishOrIrishPC,JewishPC,OtherEuropeanPC,A.agriculture.and.fishing,B.D.E.energy.and.water,C.manufacturing,F.construction,G.I.distribution..hotels.and.restaurants,H.J.transport.and.communications,K.N.banking..finance.and.insurance,O.Q.public.admin..education.and.health,R.U.other.services,G.Q.total.services,Estimated.Leave.proportion,Anomie,Public.sector,Poor.health,Distance.cosmopolian.city,Distance.university,Catholic.schools
3,0.0000,1.120997,0.222282,0.062849,0.229173,0.307998,0.177698,43.712408,92.702502,2.346535,14.558699,26.612447,22.673744,-0.059310,0.177016,0.016552,0.004273,0.896970,0.002291,0.079914,1.8,0.0,10.7,9.1,13.6,5.4,20.1,32.6,6.1,77.8,0.486547,0.266156,8.379839,3.200000,0.468253,0.141594,3.030303
61,0.4223,130.707125,0.176245,0.092353,0.390284,0.229418,0.111700,NaN,47.682345,4.547112,7.672514,31.055668,46.268415,0.454816,0.171328,0.155938,0.032507,0.491650,0.009475,0.310430,0.0,0.0,1.7,3.5,9.5,7.7,41.6,21.2,12.4,92.4,0.313504,0.474267,9.793329,4.738544,0.085459,0.011269,14.754098
78,0.2216,1.780965,0.211972,0.066947,0.214524,0.295316,0.211241,44.827941,92.603480,2.609645,16.532821,28.888551,16.681421,-0.086483,0.096221,0.011649,0.002983,0.900007,0.001804,0.083556,0.0,0.0,9.2,8.8,20.3,5.4,15.7,30.0,8.6,80.1,0.525463,0.271898,11.342854,4.106657,0.533102,0.294385,7.272727
89,0.6085,58.332540,0.231119,0.091883,0.333515,0.223124,0.120359,34.703392,53.348054,4.688395,9.289582,28.841048,28.940843,0.559536,-0.106045,0.469026,0.027242,0.351117,0.004101,0.148514,0.0,0.0,5.3,10.7,16.0,13.4,20.6,29.1,4.4,83.4,0.450562,0.315106,12.900244,4.317189,0.087567,0.087116,11.111111
97,0.2043,2.935099,0.209279,0.070123,0.245091,0.281275,0.194232,42.975704,92.427682,3.101053,15.140787,28.199268,16.772827,-0.051900,0.046128,0.034433,0.008349,0.860259,0.003023,0.093938,5.2,0.0,5.3,9.0,15.9,7.9,29.4,18.6,8.6,80.5,0.608937,0.305561,7.484035,4.118033,0.187794,0.218526,10.416667
100,0.6812,43.456031,0.262630,0.111277,0.291455,0.208921,0.125717,33.324255,70.195163,7.812335,10.162118,34.233841,12.263932,0.362468,-0.280675,0.407263,0.030972,0.485704,0.001068,0.074993,0.0,0.0,10.5,5.7,9.7,8.6,18.1,33.3,12.2,81.9,0.512173,0.304244,19.919563,6.571108,0.053657,0.018223,17.021277
102,0.2670,0.703936,0.174330,0.065370,0.190130,0.303106,0.267065,NaN,96.676472,3.584315,24.548965,39.120201,8.648622,-0.148361,-0.067355,0.005618,0.001904,0.932687,0.000833,0.058958,8.5,0.0,8.5,6.1,21.9,12.5,13.6,24.9,3.5,76.3,0.694253,0.315249,13.797587,7.821644,0.566453,0.546846,0.000000
110,0.2094,3.743381,0.224567,0.066410,0.244155,0.279673,0.185195,42.297529,91.395789,2.822986,14.127021,26.385849,21.003653,-0.057744,0.096332,0.020353,0.007233,0.876338,0.002222,0.093854,0.0,0.0,7.2,8.1,16.2,11.2,23.5,30.6,0.0,84.7,0.538611,0.269874,6.633514,3.986899,0.238725,0.229375,3.846154
117,0.5328,20.093112,0.227120,0.080843,0.270713,0.262632,0.158691,39.353506,91.005907,4.351018,13.870458,29.172431,15.644448,0.005694,-0.004288,0.060088,0.007118,0.815943,0.024526,0.092325,0.0,2.4,9.4,8.4,19.7,5.2,16.7,32.1,5.4,79.1,0.545260,0.299546,19.914000,6.071012,0.534597,0.080308,11.904762
132,0.5307,35.670314,0.199584,0.097686,0.287247,0.253631,0.161852,38.977423,89.726658,8.036619,9.630719,26.755992,8.599960,-0.076657,-0.162576,0.027717,0.007348,0.855310,0.001004,0.108621,0.0,0.0,14.7,7.2,28.2,6.2,9.9,25.2,6.9,76.4,0.678446,0.406690,20.367610,7.427708,0.534334,0.046215,3.225806


In [23]:
# Of course the demo data has a NaN in median_age
# Will just have a look at the linear model with this column ditched 
# Because it is 11pm and I don't want to write a function to estimate the missing medians from the grouped data

X = df.drop(columns=['LAB_19', 'median_age'])
y = df['LAB_19']

reg = LinearRegression().fit(X, y)
reg.score(X, y)


0.9679305912866307

In [24]:
# Good enough to drop the median_age column 
X_demo = df_all[df_all['LAB_19'].isna()].select_dtypes(exclude=['object']).drop(columns=['LAB_19', 'median_age'])
y_demo = reg.predict(X_demo)

In [25]:
y_demo
# OK getting a negative vote share for Labour is not ideal
# array([-0.04699918,  0.30239106,  0.14324523,  0.52817021,  0.12876888,
        # 0.6265624 ,  0.17511116,  0.13057389,  0.39749802,  0.40887675,
        # 0.35438663,  0.38343776,  0.29617932,  0.18009031,  0.43868441,
        # 0.39376714])

array([-0.04699918,  0.30239106,  0.14324523,  0.52817021,  0.12876888,
        0.6265624 ,  0.17511116,  0.13057389,  0.39749802,  0.40887675,
        0.35438663,  0.38343776,  0.29617932,  0.18009031,  0.43868441,
        0.39376714])

## Something more sophisticated - combine different regression models 

In [49]:
X = df.drop(columns=['LAB_19', 'median_age'])
y = df['LAB_19']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

reg1 = GradientBoostingRegressor(random_state=1)
reg2 = RandomForestRegressor(random_state=1)
reg3 = LinearRegression()

reg1.fit(X_train, y_train)
reg2.fit(X_train, y_train)
reg3.fit(X_train, y_train)

ereg = VotingRegressor([("gb", reg1), ("rf", reg2), ("lr", reg3)])
ereg.fit(X_train, y_train)

VotingRegressor(estimators=[('gb', GradientBoostingRegressor(random_state=1)),
                            ('rf', RandomForestRegressor(random_state=1)),
                            ('lr', LinearRegression())])

In [50]:
scores_unnormalised = [reg1.score(X_test, y_test), reg2.score(X_test, y_test), reg3.score(X_test, y_test), ereg.score(X_test, y_test)]
scores_unnormalised

[0.9452660298058766,
 0.9494712211628155,
 0.9453656247838753,
 0.9573718807572242]

In [51]:
# Will just try normalising the data first 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

reg1 = GradientBoostingRegressor(random_state=1)
reg2 = RandomForestRegressor(random_state=1)
reg3 = LinearRegression()

reg1.fit(X_train, y_train)
reg2.fit(X_train, y_train)
reg3.fit(X_train, y_train)

ereg = VotingRegressor([("gb", reg1), ("rf", reg2), ("lr", reg3)])
ereg.fit(X_train, y_train)

VotingRegressor(estimators=[('gb', GradientBoostingRegressor(random_state=1)),
                            ('rf', RandomForestRegressor(random_state=1)),
                            ('lr', LinearRegression())])

In [52]:
scores_normalised = [reg1.score(X_test, y_test), reg2.score(X_test, y_test), reg3.score(X_test, y_test), ereg.score(X_test, y_test)]
scores_normalised

[0.9452660298058766,
 0.9494981815395788,
 0.9453656407980259,
 0.9573814291482642]

In [53]:
# Not much improvement (if any) from normalising the data
[n - u for n, u in zip(scores_normalised, scores_unnormalised)]

[0.0, 2.6960376763307536e-05, 1.601415056118327e-08, 9.548391039948534e-06]

In [59]:
X = df.drop(columns=['LAB_19', 'median_age'])
y = df['LAB_19']

reg1.fit(X, y)
reg2.fit(X, y)
reg3.fit(X, y)

ereg = VotingRegressor([("gb", reg1), ("rf", reg2), ("lr", reg3)])
ereg.fit(X, y)

VotingRegressor(estimators=[('gb', GradientBoostingRegressor(random_state=1)),
                            ('rf', RandomForestRegressor(random_state=1)),
                            ('lr', LinearRegression())])

In [60]:
X_demo = df_all[df_all['LAB_19'].isna()].select_dtypes(exclude=['object']).drop(columns=['LAB_19', 'median_age'])
y_demo = ereg.predict(X_demo)

In [61]:
y_demo

array([0.03012892, 0.34817331, 0.15911957, 0.52055051, 0.126737  ,
       0.61884868, 0.19029174, 0.13028532, 0.43033291, 0.40041814,
       0.35596354, 0.37693923, 0.29180767, 0.18208361, 0.44354445,
       0.41135585])

## Put this data into new csv file 

In [62]:
df_all

,wpc,LAB_19,region,Winner_19,LAB_17,Winner_17,Population.density,Age_under_18,Age_18_to_24,Age_25_to_44,Age_45_to_64,Age_65_and_plus,median_age,Born.in.UK,Unemployed,Retired,Economically.Inactive,Degree,Diversity,Education,AsianPC,BlackPC,BritishOrIrishPC,JewishPC,OtherEuropeanPC,A.agriculture.and.fishing,B.D.E.energy.and.water,C.manufacturing,F.construction,G.I.distribution..hotels.and.restaurants,H.J.transport.and.communications,K.N.banking..finance.and.insurance,O.Q.public.admin..education.and.health,R.U.other.services,G.Q.total.services,Estimated.Leave.proportion,Anomie,Public.sector,Urban.rural,Poor.health,Distance.cosmopolian.city,Distance.university,Catholic.schools
0,WPC_1,0.079,South East,Conservative,0.1259,Conservative,4.023521,0.224762,0.064638,0.249319,0.270711,0.190569,41.920511,89.168905,2.484600,14.127021,26.385849,28.035015,-0.032198,0.196122,0.018034,0.003355,0.880778,0.001928,0.095905,0.0,2.6,8.9,5.3,11.3,15.3,26.4,21.6,7.4,81.9,0.406079,0.296325,7.201664,semi-rural/rural,3.263873,0.581343,0.131558,8.333333
1,WPC_2,0.077,Eastern,Conservative,0.0993,LibDem,1.163616,0.162268,0.062000,0.178540,0.298545,0.298648,51.933864,95.970229,3.369165,24.659102,37.843801,10.311081,-0.114537,-0.005407,0.004571,0.001600,0.921625,0.001043,0.071161,10.6,0.0,5.3,5.4,25.7,5.9,8.9,26.2,8.7,75.6,0.582539,0.315946,12.806969,NaN,6.331148,0.222059,0.225604,0.000000
2,WPC_3,0.670,East Midlands,Labour,0.7355,Labour,56.341334,0.229194,0.169704,0.290822,0.200038,0.110242,31.528251,66.179885,5.802527,8.017470,39.776500,19.527121,0.421376,-0.172921,0.385019,0.029043,0.476649,0.001606,0.107682,0.0,1.7,12.6,4.8,21.3,6.0,16.4,33.4,1.5,78.5,0.425820,0.357568,21.528738,Large urban,5.466648,0.756731,0.009915,4.651163
3,WPC_4,NaN,South East,Conservative,0.0000,Other,1.120997,0.222282,0.062849,0.229173,0.307998,0.177698,43.712408,92.702502,2.346535,14.558699,26.612447,22.673744,-0.059310,0.177016,0.016552,0.004273,0.896970,0.002291,0.079914,1.8,0.0,10.7,9.1,13.6,5.4,20.1,32.6,6.1,77.8,0.486547,0.266156,8.379839,semi-rural/rural,3.200000,0.468253,0.141594,3.030303
4,WPC_5,0.070,South West,Conservative,0.1153,Conservative,1.858880,0.206289,0.071584,0.212975,0.291459,0.217693,NaN,94.256623,2.884550,16.974938,30.268189,14.507132,-0.089281,0.077575,0.009133,0.002672,0.911957,0.001229,0.075008,2.3,0.0,7.4,7.2,22.4,7.0,13.6,33.9,5.4,82.4,0.534931,0.315365,9.349402,NaN,5.337837,0.528522,0.269327,5.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,WPC_339,0.363,North,Conservative,0.5338,Labour,2.292489,0.206511,0.075613,0.241952,0.290788,0.185136,42.988805,97.403092,4.321426,14.127021,26.385849,10.139870,-0.137258,-0.026257,0.006695,0.001222,0.939125,0.000714,0.052244,0.7,1.4,15.7,6.0,17.8,11.0,12.8,30.2,4.3,76.1,0.589488,0.308131,11.210694,NaN,7.995244,0.422419,0.167943,10.204082
339,WPC_340,0.548,West Midlands,Labour,0.5707,Labour,47.503505,0.265294,0.092883,0.280616,0.219427,0.141780,NaN,82.101032,7.286231,11.716791,33.755743,9.220723,0.123208,-0.306308,0.247038,0.012456,0.672756,0.001066,0.066683,0.0,0.0,6.0,8.5,19.5,15.0,13.7,30.5,4.8,83.4,0.600948,0.277969,13.496404,Large urban,7.157713,0.071343,0.085990,12.765957
340,WPC_341,0.449,Wales,Labour,0.5446,Labour,7.593529,0.226013,0.082795,0.267612,0.264868,0.158713,39.481845,97.255691,5.045756,15.635712,34.608007,11.392369,-0.130021,-0.026680,0.009527,0.001855,0.932375,0.001157,0.055086,0.0,3.3,12.8,7.1,16.8,4.4,14.4,34.0,5.7,75.3,0.551576,0.282654,24.855058,NaN,9.005518,0.101090,0.111517,2.272727
341,WPC_342,0.533,North West,Labour,0.5916,Labour,7.256798,0.202000,0.079753,0.243704,0.286358,0.188184,42.851385,95.915456,3.958525,19.469748,30.050492,12.496144,-0.132450,-0.019418,0.010168,0.002431,0.924005,0.001616,0.061780,0.0,0.0,13.0,7.3,23.8,9.5,17.5,20.6,5.6,76.9,0.577668,0.285594,10.589866,Large urban,6.134871,0.591412,0.085013,16.216216


In [66]:
df_all["LAB_19"] = df_all["LAB_19"] * 100  # Original data had percentage for LAB_19 so convert back
df_all.loc[df_all["LAB_19"].isna(), "LAB_19"] = y_demo * 100b

ValueError: Must have equal len keys and value when setting with an iterable

In [70]:
df_all["LAB_19"] = df_all["LAB_19"].round(decimals=1) 

In [71]:
df_all

,wpc,LAB_19,region,Winner_19,LAB_17,Winner_17,Population.density,Age_under_18,Age_18_to_24,Age_25_to_44,Age_45_to_64,Age_65_and_plus,median_age,Born.in.UK,Unemployed,Retired,Economically.Inactive,Degree,Diversity,Education,AsianPC,BlackPC,BritishOrIrishPC,JewishPC,OtherEuropeanPC,A.agriculture.and.fishing,B.D.E.energy.and.water,C.manufacturing,F.construction,G.I.distribution..hotels.and.restaurants,H.J.transport.and.communications,K.N.banking..finance.and.insurance,O.Q.public.admin..education.and.health,R.U.other.services,G.Q.total.services,Estimated.Leave.proportion,Anomie,Public.sector,Urban.rural,Poor.health,Distance.cosmopolian.city,Distance.university,Catholic.schools
0,WPC_1,7.9,South East,Conservative,0.1259,Conservative,4.023521,0.224762,0.064638,0.249319,0.270711,0.190569,41.920511,89.168905,2.484600,14.127021,26.385849,28.035015,-0.032198,0.196122,0.018034,0.003355,0.880778,0.001928,0.095905,0.0,2.6,8.9,5.3,11.3,15.3,26.4,21.6,7.4,81.9,0.406079,0.296325,7.201664,semi-rural/rural,3.263873,0.581343,0.131558,8.333333
1,WPC_2,7.7,Eastern,Conservative,0.0993,LibDem,1.163616,0.162268,0.062000,0.178540,0.298545,0.298648,51.933864,95.970229,3.369165,24.659102,37.843801,10.311081,-0.114537,-0.005407,0.004571,0.001600,0.921625,0.001043,0.071161,10.6,0.0,5.3,5.4,25.7,5.9,8.9,26.2,8.7,75.6,0.582539,0.315946,12.806969,NaN,6.331148,0.222059,0.225604,0.000000
2,WPC_3,67.0,East Midlands,Labour,0.7355,Labour,56.341334,0.229194,0.169704,0.290822,0.200038,0.110242,31.528251,66.179885,5.802527,8.017470,39.776500,19.527121,0.421376,-0.172921,0.385019,0.029043,0.476649,0.001606,0.107682,0.0,1.7,12.6,4.8,21.3,6.0,16.4,33.4,1.5,78.5,0.425820,0.357568,21.528738,Large urban,5.466648,0.756731,0.009915,4.651163
3,WPC_4,3.0,South East,Conservative,0.0000,Other,1.120997,0.222282,0.062849,0.229173,0.307998,0.177698,43.712408,92.702502,2.346535,14.558699,26.612447,22.673744,-0.059310,0.177016,0.016552,0.004273,0.896970,0.002291,0.079914,1.8,0.0,10.7,9.1,13.6,5.4,20.1,32.6,6.1,77.8,0.486547,0.266156,8.379839,semi-rural/rural,3.200000,0.468253,0.141594,3.030303
4,WPC_5,7.0,South West,Conservative,0.1153,Conservative,1.858880,0.206289,0.071584,0.212975,0.291459,0.217693,NaN,94.256623,2.884550,16.974938,30.268189,14.507132,-0.089281,0.077575,0.009133,0.002672,0.911957,0.001229,0.075008,2.3,0.0,7.4,7.2,22.4,7.0,13.6,33.9,5.4,82.4,0.534931,0.315365,9.349402,NaN,5.337837,0.528522,0.269327,5.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,WPC_339,36.3,North,Conservative,0.5338,Labour,2.292489,0.206511,0.075613,0.241952,0.290788,0.185136,42.988805,97.403092,4.321426,14.127021,26.385849,10.139870,-0.137258,-0.026257,0.006695,0.001222,0.939125,0.000714,0.052244,0.7,1.4,15.7,6.0,17.8,11.0,12.8,30.2,4.3,76.1,0.589488,0.308131,11.210694,NaN,7.995244,0.422419,0.167943,10.204082
339,WPC_340,54.8,West Midlands,Labour,0.5707,Labour,47.503505,0.265294,0.092883,0.280616,0.219427,0.141780,NaN,82.101032,7.286231,11.716791,33.755743,9.220723,0.123208,-0.306308,0.247038,0.012456,0.672756,0.001066,0.066683,0.0,0.0,6.0,8.5,19.5,15.0,13.7,30.5,4.8,83.4,0.600948,0.277969,13.496404,Large urban,7.157713,0.071343,0.085990,12.765957
340,WPC_341,44.9,Wales,Labour,0.5446,Labour,7.593529,0.226013,0.082795,0.267612,0.264868,0.158713,39.481845,97.255691,5.045756,15.635712,34.608007,11.392369,-0.130021,-0.026680,0.009527,0.001855,0.932375,0.001157,0.055086,0.0,3.3,12.8,7.1,16.8,4.4,14.4,34.0,5.7,75.3,0.551576,0.282654,24.855058,NaN,9.005518,0.101090,0.111517,2.272727
341,WPC_342,53.3,North West,Labour,0.5916,Labour,7.256798,0.202000,0.079753,0.243704,0.286358,0.188184,42.851385,95.915456,3.958525,19.469748,30.050492,12.496144,-0.132450,-0.019418,0.010168,0.002431,0.924005,0.001616,0.061780,0.0,0.0,13.0,7.3,23.8,9.5,17.5,20.6,5.6,76.9,0.577668,0.285594,10.589866,Large urban,6.134871,0.591412,0.085013,16.216216


In [72]:
df_all.to_csv('data_out.csv', index=False)